# f) Ngonye Falls Power and Energy Periods



## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|



## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|



## Parameters

In [145]:
input_data='./input_data/'
output_data='./output_data/'

In [146]:
out_prefix='rec1'
debt_period=18
equity_period=25
discount_rate=1.0 + 0.15

## Libraries and Setup

In [147]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

In [148]:
yearly=pd.read_csv(output_data + out_prefix + "_p_and_e_yearly.csv")

In [149]:
yearly

,WaterYear,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor,Prop4Units,Prop3Units,Prop2Units,Prop1Units,Prop0Units,Days0Units
0,1924,3456.928400,111.133124,0.872827,0.052937,25.408305,13.482746,974.580352,964.401972,189.887819,0.000000,827686.134030,0.524915,0.408219,0.035616,0.109589,0.419178,0.027397,10
1,1925,4504.008836,157.205464,0.566039,0.040630,25.355414,11.551136,976.511962,964.401972,193.747347,22.358325,767292.232182,0.486614,0.317808,0.076712,0.189041,0.416438,0.000000,0
2,1926,3305.305419,192.534369,0.496243,0.065652,25.437234,13.793090,974.270008,964.401972,190.872868,20.757348,857958.191273,0.544114,0.378082,0.060274,0.178082,0.383562,0.000000,0
3,1927,2256.674509,178.800013,0.473067,0.065583,25.390386,16.233306,971.829792,964.401972,189.332151,22.514464,826892.270682,0.524412,0.304110,0.106849,0.178082,0.413699,0.000000,0
4,1928,1587.035405,195.867340,0.502447,0.093256,25.387509,18.166094,969.897004,964.405963,192.801268,22.924151,747056.070205,0.473780,0.172603,0.145205,0.210959,0.471233,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2012,3617.058785,224.058948,0.505204,0.055725,25.076798,13.164406,974.898692,964.717318,191.420746,24.932292,843394.858484,0.534878,0.413699,0.076712,0.128767,0.380822,0.000000,0
89,2013,3578.198961,194.855402,0.497805,0.056469,25.437511,13.240804,974.822294,964.400513,191.638019,20.627822,926765.634148,0.587751,0.446575,0.131507,0.128767,0.293151,0.000000,0
90,2014,1460.437582,205.608866,0.542968,0.100587,25.105646,18.622820,969.485125,964.706250,192.445122,24.641825,885184.222151,0.561380,0.306849,0.117808,0.161644,0.413699,0.000000,0
91,2015,2973.367833,173.346774,0.501598,0.067263,25.390627,14.505749,973.557349,964.400513,191.219733,21.113961,787616.850388,0.499503,0.326027,0.068493,0.169863,0.438356,0.000000,0


In [150]:
sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims=sims.set_index('StartYear')
sims

,DebtEnd,EquityEnd
StartYear,,
1924,1941,1948
1925,1942,1949
1926,1943,1950
1927,1944,1951
1928,1945,1952
...,...,...
1988,2005,2012
1989,2006,2013
1990,2007,2014


In [151]:


o=pd.DataFrame()
for row in sims.itertuples(): 
    f=pd.DataFrame(yearly.loc[(yearly['WaterYear']>=row.Index) & (yearly['WaterYear']<=row.EquityEnd)]['Energy'])
    f['Energy']=f['Energy']/1000
    f=f.reset_index()
    f['YearNo']=f.index+1
    f['StartYear']=row.Index
    f=f.set_index('StartYear').drop('index',axis=1)
    #display(f)
    
    o=o.append(f)

sims=sims.join(o)
#sims=sims.reset_index()
#sims=sims.set_index(['StartYear','YearNo'])
sims
#o

,DebtEnd,EquityEnd,Energy,YearNo
StartYear,,,,
1924,1941,1948,827.686134,1
1924,1941,1948,767.292232,2
1924,1941,1948,857.958191,3
1924,1941,1948,826.892271,4
1924,1941,1948,747.056070,5
...,...,...,...,...
1992,2009,2016,843.394858,21
1992,2009,2016,926.765634,22
1992,2009,2016,885.184222,23


In [152]:
sims['CumEnergy']=sims['Energy'].groupby('StartYear').cumsum()

In [153]:
sims['DiscountedEnergy']=sims['Energy']*(1/(discount_rate**(sims['YearNo']-1)))

In [154]:
sims['CumDiscountedEnergy']=sims['DiscountedEnergy'].groupby('StartYear').cumsum()

In [159]:
sims

,DebtEnd,EquityEnd,Energy,YearNo,CumEnergy,DiscountedEnergy,CumDiscountedEnergy
StartYear,,,,,,,
1924,1941,1948,827.686134,1,827.686134,827.686134,827.686134
1924,1941,1948,767.292232,2,1594.978366,667.210637,1494.896771
1924,1941,1948,857.958191,3,2452.936557,648.739653,2143.636424
1924,1941,1948,826.892271,4,3279.828828,543.695090,2687.331514
1924,1941,1948,747.056070,5,4026.884898,427.131733,3114.463247
...,...,...,...,...,...,...,...
1992,2009,2016,843.394858,21,16673.648519,51.531661,5340.122294
1992,2009,2016,926.765634,22,17600.414153,49.239686,5389.361980
1992,2009,2016,885.184222,23,18485.598376,40.896032,5430.258012


In [155]:
sims_summary=(sims.loc[sims['YearNo']==debt_period]).drop(['Energy','YearNo','DiscountedEnergy','DebtEnd','EquityEnd'],axis=1).rename(columns={'CumEnergy':'DebtEnergy','CumDiscountedEnergy':'DebtDiscountedEnergy'})

sims_summary=sims_summary.join((sims.loc[sims['YearNo']==equity_period])
            .drop(['Energy','YearNo','DiscountedEnergy'],axis=1)
            .rename(columns={'CumEnergy':'EquityEnergy','CumDiscountedEnergy':'EquityDiscountedEnergy'}))
sims_summary.head(3)

,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy
StartYear,,,,,,
1924,14843.198628,5776.958973,1941,1948,20390.234536,6087.704010
1925,14824.139120,5766.806111,1942,1949,20317.085109,6075.379756
1926,14902.268031,5828.002470,1943,1950,20395.091677,6133.830559


In [156]:
sims_summary['DebtAvEnergy']=sims.loc[sims['YearNo']<=debt_period]['Energy'].groupby('StartYear').mean()

In [157]:
sims_summary['EquityAvEnergy']=sims.loc[sims['YearNo']<=equity_period]['Energy'].groupby('StartYear').mean()

In [158]:
sims_summary

,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy
StartYear,,,,,,,,
1924,14843.198628,5776.958973,1941,1948,20390.234536,6087.704010,824.622146,815.609381
1925,14824.139120,5766.806111,1942,1949,20317.085109,6075.379756,823.563284,812.683404
1926,14902.268031,5828.002470,1943,1950,20395.091677,6133.830559,827.903779,815.803667
1927,14840.219516,5789.511533,1944,1951,20343.518819,6095.423716,824.456640,813.740753
1928,14884.444220,5787.961469,1945,1952,20331.138019,6087.265537,826.913568,813.245521
...,...,...,...,...,...,...,...,...
1988,13695.418851,5256.850621,2005,2012,19732.680434,5589.486747,760.856603,789.307217
1989,13689.713803,5154.656709,2006,2013,19817.318400,5491.838833,760.539656,792.692736
1990,13833.298848,5150.275743,2007,2014,19954.284281,5486.086841,768.516603,798.171371


In [143]:
sims_summary.to_csv(output_data + out_prefix + "_p_and_e_sims_summary.csv")

In [162]:
sims.to_csv(output_data + out_prefix + "_p_and_e_sims.csv")